In [197]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
import requests
import warnings

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [198]:
races = pd.read_csv(path+'data/races.csv')
results = pd.read_csv(path+'data/results.csv')

weather = races.iloc[:,[0,1,2,3,4,5,7]]
weather = weather.query('season == 2021')
weather.shape

(22, 7)

In [199]:
def format_datetime(dt):
    year, time = dt.split(' ')

    return datetime(*map(int, year.split('-')), *map(int, time.split(':')))

In [200]:
weather.date = weather.date.apply(lambda x: format_datetime(x))

In [217]:
def get_hourly_weather(dt, lat, long):
    start = dt
    end = dt + pd.Timedelta(hours=3)
    coord = Point(lat, long)

    data = Hourly(coord, start, end).fetch()

    return data.rhum.values

def get_daily_weather(dt, lat, long):
    start = dt
    end = dt
    coord = Point(lat, long)

    data = Daily(coord, start, end).fetch()

    return data.coco.values

In [218]:
weather['code'] = weather.apply(lambda x: get_hourly_weather(x['date'], x['lat'], x['long']),axis=1)

In [220]:
weather.head()

,season,round,circuit_id,country,lat,long,date,code
138,2021,1,bahrain,Bahrain,26.0325,50.51060,2021-03-28 15:00:00,"[57.0, 60.0, 53.0, 53.0]"
139,2021,2,imola,Italy,44.3439,11.71670,2021-04-18 13:00:00,"[47.0, 47.0, 47.0, 47.0]"
140,2021,3,portimao,Portugal,37.2270,-8.62670,2021-05-02 14:00:00,[]
141,2021,4,catalunya,Spain,41.5700,2.26111,2021-05-09 13:00:00,"[64.0, 73.0, 61.0, 73.0]"
142,2021,5,monaco,Monaco,43.7347,7.42056,2021-05-23 13:00:00,"[41.0, 41.0, 43.0, 45.0]"


In [ ]:
def format_coco(coco):
    codes = {
        1.0 :'Clear',
        2.0 : 'Fair',
        3.0 : 'Cloudy',
        4.0 : 'Overcast',
        5.0 : 'Fog',
        6.0 : 'Freezing Fog',
        7.0 : 'Light Rain',
        8.0 : 'Rain',
        9.0 : 'Heavy Rain',
        10.0 : 'Freezing Rain',
        11.0 : 'Heavy Freezing Rain',
        12.0 : 'Sleet',
        13.0 : 'Heavy Sleet',
        14.0 : 'Light Snowfall',
        15.0 : 'Snowfall',
        16.0 : 'Heavy Snowfall',
        17.0 : 'Rain Shower',
        18.0 : 'Heavy Rain Shower',
        19.0 : 'Sleet Shower',
        20.0 : 'Heavy Sleet Shower',
        21.0 : 'Snow Shower',
        22.0 : 'Heavy Snow Shower',
        23.0 : 'Lightning',
        24.0 : 'Hail',
        25.0 : 'Thunderstorm',
        26.0 : 'Heavy Thunderstorm',
        27.0 : 'Storm',
    }
    return codes[coco]

In [ ]:
# start = datetime.strptime(race.date, '%Y-%m-%d')
# end = datetime.strptime(race.date, '%Y-%m-%d')
start = datetime(2021, 6, 27)
end = datetime(2021, 6, 27,23)
coord = Point(47.2197, 14.7647)

data = Hourly(coord, start, end).fetch()

# weather = data.groupby('coco').rhum.describe().reset_index()
# weather['coco'] = weather.coco.apply(lambda x: format_coco(x))